In [2]:
import string
import nltk
import re
from tqdm import tnrange, tqdm_notebook

import pandas as pd
from collections import Counter
from nltk.corpus import stopwords
import numpy as np

In [10]:
stop_words = [
    'и', 'в', 'во', 'не', 'что', 'на', 'с', 'со', 'как', 'а', 'то', 'все', 'так', 'его', 'но', 'да', 'к',
    'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о','у', 'же', 'вы', 'за', 'бы',
    'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о','из', 'ему', 'теперь',
    'когда', 'даже', 'ну', 'вдруг', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже',
    'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был',
    'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'ведь', 'там', 'потом', 'себя',
    'ничего', 'ей', 'может', 'они', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они',
    'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто',
    'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого',
    'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас',
    'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после',
    'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту',
    'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им',
    'более', 'всегда', 'конечно', 'всю', 'между', 'что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', '--', 'к',
    'на', '...', '``', 'о-о', '–', "''", 'и', 'мог', 'он', 'я', 'ты','она','они'
]

def print_dictionaries(default_dict, processed_dict, deleted_indexes):
    print("Corpus length BEFORE processing: " + str(len(default_dict)))
    print("Corpus length AFTER processing: " + str(len(final_dict)))
    print("Corpus diff length: " + str(len(default_dict) - len(final_dict)))
    print("\n")

    print("Default corpus:")
    for a in default_dict.most_common(20):
        print (a[0] + ' : ' + str(a[1]))

    print("\n")
    print("Processed corpus:")
    for a in processed_dict.most_common(20):
        print (a[0] + ' : ' + str(a[1]))

    print("\n\n")
    print("Deleted indexes:")
    for a in deleted_indexes:
        print (a[0] + ' : ' + str(a[1]))
        

#     print("\n\n")
#     print("По алфавиту:")
#     for a in final_dict:
#         print (a)


In [4]:
def levenshtein(a, b):
    n, m = len(a), len(b)
    if n > m:
        a, b = b, a
        n, m = m, n

    current_row = range(n+1)
    for i in range(1, m+1):
        previous_row, current_row = current_row, [i]+[0]*n

        for j in range(1,n+1):
            add, delete, change = previous_row[j]+1, current_row[j-1]+1, previous_row[j-1]
            if a[j-1] != b[i-1]:
                change += 1
            current_row[j] = min(add, delete, change)
    return current_row[n]

## Разбить текст на токены
def tokenize(file_text):
    tokens = re.findall(r"[\w']+|[.,!?;]", file_text)
    tokens = [i.lower() for i in tokens if (i not in string.punctuation)]
    tokens = [i for i in tokens if (i not in stop_words)]
    tokens = [i.replace("«", "").replace("»", "") for i in tokens]

    return tokens

In [5]:
file_content = open('data/starik_i_more.txt', 'r', encoding='utf-8').read()

# Заменить цифры пробелами
file_content = re.sub(r"\d+", ' ', file_content)
tokens = tokenize(file_content)

# Отсортировать токены в алфавитном порядке
tokens = sorted(tokens)
token_counter = Counter(tokens)
indexes = token_counter.keys()

#### Расчет матрицы схожести занял в районе 16 минут

In [6]:
matrix = np.matrix([[levenshtein(y, x) for x in indexes] for y in tqdm_notebook(indexes)])

# создаем датафрейм
df = pd.DataFrame(matrix, 
                  columns=indexes, 
                  index=indexes)

df

,brisa,calambre,campeon,cobre,cпортивные,de,del,dentuso,dorado,el,...,ярком,яростной,яростную,ярче,ясная,ясно,ясной,ястребов,ястребы,ящик
brisa,0,8,7,5,10,5,5,6,5,5,...,5,8,8,5,5,5,5,8,7,5
calambre,8,0,6,4,9,7,7,8,7,7,...,8,8,8,8,8,8,8,8,8,8
campeon,7,6,0,5,9,6,6,7,6,6,...,7,8,8,7,7,7,7,8,7,7
cobre,5,4,5,0,9,4,5,7,5,5,...,5,8,8,5,5,5,5,8,7,5
cпортивные,10,9,9,9,0,10,10,10,10,10,...,9,7,7,8,9,9,9,9,8,9
de,5,7,6,4,10,0,1,5,5,2,...,5,8,8,4,5,4,5,8,7,4
del,5,7,6,5,10,1,0,5,5,1,...,5,8,8,4,5,4,5,8,7,4
dentuso,6,8,7,7,10,5,5,0,5,6,...,7,8,8,7,7,7,7,8,7,7
dorado,5,7,6,5,10,5,5,5,0,6,...,6,8,8,6,6,6,6,8,7,6
el,5,7,6,5,10,2,1,6,6,0,...,5,8,8,4,5,4,5,8,7,4


In [29]:
df.to_csv('data/levenshtain.csv')

##### Примем слова с разной первой буквой за разные слова

In [11]:
default_dict = token_counter.copy()
final_dict = default_dict.copy()

deleted_indexes = []
indexes = list(token_counter)

corpus_length = len(indexes)
# Для экономии времени сделал предположение, что в отсортированном массиве за пять проходов я смогу слить похожие слова.
for _ in tqdm_notebook(range(5)):
    indexes = list(final_dict)

    for i in range(len(indexes) - 1):
        first_word, second_word = indexes[i], indexes[i + 1]

        if (len(first_word) < 4):
            continue

        # допустим слова с разной первой буквой - разные слова
        if first_word[0] == second_word[0]:
            if df[first_word][second_word] == 1 and first_word[-1] != second_word[-1]:
                final_dict[first_word] += default_dict[second_word]
                del(final_dict[second_word])
                deleted_indexes.append((first_word, second_word))
            elif df[first_word][second_word] == 2 and len(first_word) > 4:
                if (first_word[-2:] != second_word[-2:]):
                    final_dict[first_word] += default_dict[second_word]
                    del(final_dict[second_word])
                    deleted_indexes.append((first_word, second_word))
            elif df[first_word][second_word] == 3 and len(first_word) > 6:
                if (first_word[-2:] != second_word[-2:]):
                    final_dict[first_word] += default_dict[second_word]
                    del(final_dict[second_word])
                    deleted_indexes.append((first_word, second_word))




print_dictionaries(default_dict, final_dict, deleted_indexes)


Corpus length BEFORE processing: 5224
Corpus length AFTER processing: 3627
Corpus diff length: 1597


Default corpus:
старик : 293
сказал : 146
рыба : 134
рыбу : 87
снова : 76
подумал : 75
мальчик : 70
рыбы : 70
море : 55
воду : 41
очень : 41
лодки : 39
тебе : 38
руки : 37
лесу : 36
руку : 35
солнце : 35
думал : 34
лодку : 32
акула : 31


Processed corpus:
старик : 315
рыба : 155
подумай : 150
сказал : 148
рыбой : 87
снова : 82
мальчик : 78
лодке : 78
думай : 68
море : 58
виде : 52
солнца : 48
водою : 42
акул : 41
очень : 41
лодках : 40
тебе : 38
вода : 36
стал : 36
гладь : 36



Deleted indexes:
galano : galanos
акул : акула
акула : акулам
акулам : акулами
акулами : акуле
акуле : акулой
акулой : акулу
акулу : акулы
акульей : акульи
акульи : акулья
амальгаму : амальгамы
африка : африке
африке : африки
багор : багром
банку : банок
бедром : бедру
бейсбол : бейсболе
бейсбольная : бейсбольных
белое : белой
белой : белую
белую : белые
белые : белый
белый : белым
белым : белыми
белыми : бел

поранил : поранила
породы : порой
порой : порою
порою : порт
послушай : послушал
поступит : поступить
потер : потерев
потеряв : потеряет
похожа : похоже
похоже : похожи
похожи : похожие
почувствовал : почувствовала
правда : правде
правде : правду
править : право
право : правого
правого : правое
правое : правой
правой : правую
прежнее : прежней
прежней : прежнем
прежнем : прежнему
прежнему : прежние
прежние : прежний
прекрасная : прекрасно
привезла : привязав
привязав : привязал
привязал : привязано
придумай : придут
прижался : прижатая
приманкой : приманку
принести : принесу
принесу : принеся
причиняет : причиняла
причиняла : причинять
пришелся : пришла
пришла : пришлась
пришлась : пришло
проверить : проверь
проволоке : проволокой
продолжала : продолжать
проплывали : проплывая
просил : просишь
просишь : проснись
проснись : прости
противна : противнее
противнее : противника
протухла : протухли
прочна : прочту
прочту : прочтя
прошла : прошли
прошли : прошло
прошло : прошлом
прошлом : про

In [8]:
default_corpus_length = len(tokens)
final_indexes = []
word_frequencies = []

for el in final_dict.most_common():
    final_indexes.append(el[0])
    word_frequencies.append(el[1] / default_corpus_length)

frequency_df = pd.DataFrame([word_frequencies], columns=final_indexes)
frequency_df.to_csv('data/result.csv')   
frequency_df
    

,старик,рыба,подумай,сказал,рыбой,снова,мальчик,лодке,думай,море,...,весло,голод,делало,делать,другими,запасных,острогу,острое,плавник,рыбаку
0,0.024866,0.012236,0.011841,0.011683,0.006868,0.006473,0.006157,0.006157,0.005368,0.004578,...,0.000079,0.000079,0.000079,0.000079,0.000079,0.000079,0.000079,0.000079,0.000079,0.000079
